In [118]:
sc.applicationId

'local-1698763673578'

In [114]:
dfs = spark.read.parquet('./03_Feature_Engineering_end.parquet')
dfs.show(5,0)
dfs.printSchema()


+-----------------------------+----------+----+-----+-------------------+------------------+----------+---------------+-------------+---------------+----------+---------+---------------+-----------------+------------+--------------+-------+---------+-----------------+------------+--------------+----------+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+----------+------------------+---------------+----+--------------------------------------------------------------------------+---------------+-------+----------+---------+------------------+-------+----------+----------+---------+------------------+----------------+-----+------------------+----------------------------------------------------------------------------------------------------------------------------------------

In [119]:
dfs.count()

6090

In [103]:
from datetime import timedelta, datetime

def train_test_split_date(df, split_col, test_days=45):
    """Calculate the date to split test and training sets"""
    # Find how many days our data spans
    # datetime.strptime(split_date, '%Y-%m-%d')
    max_date = datetime.strptime(df.agg({split_col: 'max'}).collect()[0][0], '%Y-%m-%d')
    min_date = datetime.strptime(df.agg({split_col: 'min'}).collect()[0][0], '%Y-%m-%d')
    # Subtract an integer number of days from the last date in dataset
    split_date = max_date - timedelta(days=test_days)
    return split_date

df = spark.read.csv('./tmp_test_m_5000.csv',header=True, inferSchema=True)
# Find the date to use in spitting test and train
split_date = train_test_split_date(df, 'OFFMKTDATE')

# Create Sequential Test and Training Sets
train_df = df.where(df['OFFMKTDATE'] < split_date)
test_df = df\
    .where(df['OFFMKTDATE'] >= split_date)\
    .where(df['LISTDATE'] <= split_date) 

In [109]:
df.describe().show()

+-------+-----------------+----------+----------+
|summary|     DAYSONMARKET|  LISTDATE|OFFMKTDATE|
+-------+-----------------+----------+----------+
|  count|             5000|      5000|      5000|
|   mean|         111.2658|      NULL|      NULL|
| stddev|64.57408700789027|      NULL|      NULL|
|    min|                0|2017-07-10|2017-07-11|
|    max|              224|2017-10-25|2017-10-31|
+-------+-----------------+----------+----------+



In [107]:
test_df.count()
train_df.count()

3010

In [111]:
from pyspark.sql.functions import datediff, to_date, lit

split_date = to_date(lit('2017-09-10'))
# Create Sequential Test set
test_df = df\
    .where(df['OFFMKTDATE'] >= split_date)\
    .where(df['LISTDATE'] <= split_date)

# Create a copy of DAYSONMARKET to review later
test_df = test_df.withColumn('DAYSONMARKET_Original', test_df['DAYSONMARKET'])

# Recalculate DAYSONMARKET from what we know on our split date
test_df = test_df.withColumn('DAYSONMARKET', datediff(split_date, df['LISTDATE']))

# Review the difference
test_df[['LISTDATE', 'OFFMKTDATE', 'DAYSONMARKET_Original', 'DAYSONMARKET']].show(5)

+----------+----------+---------------------+------------+
|  LISTDATE|OFFMKTDATE|DAYSONMARKET_Original|DAYSONMARKET|
+----------+----------+---------------------+------------+
|2017-09-09|2017-09-14|                   87|           1|
|2017-09-09|2017-09-10|                  125|           1|
|2017-09-10|2017-09-11|                   65|           0|
|2017-09-07|2017-09-12|                  216|           3|
|2017-09-06|2017-09-10|                   40|           4|
+----------+----------+---------------------+------------+
only showing top 5 rows



In [112]:
binary_cols = ['FENCE_WIRE', 'FENCE_ELECTRIC', 'FENCE_NAN', 'FENCE_PARTIAL', 'FENCE_RAIL', 'FENCE_OTHER', 'FENCE_CHAIN LINK', 'FENCE_FULL', 'FENCE_NONE', 'FENCE_PRIVACY', 'FENCE_WOOD', 'FENCE_INVISIBLE', 'ROOF_ASPHALT SHINGLES', 'ROOF_SHAKES', 'ROOF_NAN', 'ROOF_UNSPECIFIED SHINGLE', 'ROOF_SLATE', 'ROOF_PITCHED', 'ROOF_FLAT', 'ROOF_TAR/GRAVEL', 'ROOF_OTHER', 'ROOF_METAL', 'ROOF_TILE', 'ROOF_RUBBER', 'ROOF_WOOD SHINGLES', 'ROOF_AGE OVER 8 YEARS', 'ROOF_AGE 8 YEARS OR LESS', 'POOLDESCRIPTION_NAN', 'POOLDESCRIPTION_HEATED', 'POOLDESCRIPTION_NONE', 'POOLDESCRIPTION_SHARED', 'POOLDESCRIPTION_INDOOR', 'POOLDESCRIPTION_OUTDOOR', 'POOLDESCRIPTION_ABOVE GROUND', 'POOLDESCRIPTION_BELOW GROUND', 'GARAGEDESCRIPTION_ASSIGNED', 'GARAGEDESCRIPTION_TANDEM', 'GARAGEDESCRIPTION_UNCOVERED/OPEN', 'GARAGEDESCRIPTION_TUCKUNDER', 'GARAGEDESCRIPTION_DRIVEWAY - ASPHALT', 'GARAGEDESCRIPTION_HEATED GARAGE', 'GARAGEDESCRIPTION_UNDERGROUND GARAGE', 'GARAGEDESCRIPTION_DRIVEWAY - SHARED', 'GARAGEDESCRIPTION_CONTRACT PKG REQUIRED', 'GARAGEDESCRIPTION_GARAGE DOOR OPENER', 'GARAGEDESCRIPTION_MORE PARKING OFFSITE FOR FEE', 'GARAGEDESCRIPTION_VALET PARKING FOR FEE', 'GARAGEDESCRIPTION_OTHER', 'GARAGEDESCRIPTION_MORE PARKING ONSITE FOR FEE', 'GARAGEDESCRIPTION_DRIVEWAY - OTHER SURFACE', 'GARAGEDESCRIPTION_DETACHED GARAGE', 'GARAGEDESCRIPTION_SECURED', 'GARAGEDESCRIPTION_CARPORT', 'GARAGEDESCRIPTION_DRIVEWAY - CONCRETE', 'GARAGEDESCRIPTION_ON-STREET PARKING ONLY', 'GARAGEDESCRIPTION_COVERED', 'GARAGEDESCRIPTION_INSULATED GARAGE', 'GARAGEDESCRIPTION_UNASSIGNED', 'GARAGEDESCRIPTION_NONE', 'GARAGEDESCRIPTION_DRIVEWAY - GRAVEL', 'GARAGEDESCRIPTION_NO INT ACCESS TO DWELLING', 'GARAGEDESCRIPTION_UNITS VARY', 'GARAGEDESCRIPTION_ATTACHED GARAGE', 'APPLIANCES_NAN', 'APPLIANCES_COOKTOP', 'APPLIANCES_WALL OVEN', 'APPLIANCES_WATER SOFTENER - OWNED', 'APPLIANCES_DISPOSAL', 'APPLIANCES_DISHWASHER', 'APPLIANCES_OTHER', 'APPLIANCES_INDOOR GRILL', 'APPLIANCES_WASHER', 'APPLIANCES_RANGE', 'APPLIANCES_REFRIGERATOR', 'APPLIANCES_FURNACE HUMIDIFIER', 'APPLIANCES_TANKLESS WATER  HEATER', 'APPLIANCES_ELECTRONIC AIR FILTER', 'APPLIANCES_MICROWAVE', 'APPLIANCES_EXHAUST FAN/HOOD', 'APPLIANCES_NONE', 'APPLIANCES_CENTRAL VACUUM', 'APPLIANCES_TRASH COMPACTOR', 'APPLIANCES_AIR-TO-AIR EXCHANGER', 'APPLIANCES_DRYER', 'APPLIANCES_FREEZER', 'APPLIANCES_WATER SOFTENER - RENTED', 'EXTERIOR_SHAKES', 'EXTERIOR_CEMENT BOARD', 'EXTERIOR_BLOCK', 'EXTERIOR_VINYL', 'EXTERIOR_FIBER BOARD', 'EXTERIOR_OTHER', 'EXTERIOR_METAL', 'EXTERIOR_BRICK/STONE', 'EXTERIOR_STUCCO', 'EXTERIOR_ENGINEERED WOOD', 'EXTERIOR_WOOD', 'DININGROOMDESCRIPTION_EAT IN KITCHEN', 'DININGROOMDESCRIPTION_NAN', 'DININGROOMDESCRIPTION_OTHER', 'DININGROOMDESCRIPTION_LIVING/DINING ROOM', 'DININGROOMDESCRIPTION_SEPARATE/FORMAL DINING ROOM', 'DININGROOMDESCRIPTION_KITCHEN/DINING ROOM', 'DININGROOMDESCRIPTION_INFORMAL DINING ROOM', 'DININGROOMDESCRIPTION_BREAKFAST AREA', 'BASEMENT_FINISHED (LIVABLE)', 'BASEMENT_PARTIAL', 'BASEMENT_SUMP PUMP', 'BASEMENT_INSULATING CONCRETE FORMS', 'BASEMENT_CRAWL SPACE', 'BASEMENT_PARTIAL FINISHED', 'BASEMENT_CONCRETE BLOCK', 'BASEMENT_DRAINAGE SYSTEM', 'BASEMENT_POURED CONCRETE', 'BASEMENT_UNFINISHED', 'BASEMENT_DRAIN TILED', 'BASEMENT_WOOD', 'BASEMENT_FULL', 'BASEMENT_EGRESS WINDOWS', 'BASEMENT_DAY/LOOKOUT WINDOWS', 'BASEMENT_SLAB', 'BASEMENT_STONE', 'BASEMENT_NONE', 'BASEMENT_WALKOUT', 'BATHDESC_MAIN FLOOR 1/2 BATH', 'BATHDESC_TWO MASTER BATHS', 'BATHDESC_MASTER WALK-THRU', 'BATHDESC_WHIRLPOOL', 'BATHDESC_NAN', 'BATHDESC_3/4 BASEMENT', 'BATHDESC_TWO BASEMENT BATHS', 'BATHDESC_OTHER', 'BATHDESC_3/4 MASTER', 'BATHDESC_MAIN FLOOR 3/4 BATH', 'BATHDESC_FULL MASTER', 'BATHDESC_MAIN FLOOR FULL BATH', 'BATHDESC_WALK-IN SHOWER', 'BATHDESC_SEPARATE TUB & SHOWER', 'BATHDESC_FULL BASEMENT', 'BATHDESC_BASEMENT', 'BATHDESC_WALK THRU', 'BATHDESC_BATHROOM ENSUITE', 'BATHDESC_PRIVATE MASTER', 'BATHDESC_JACK & JILL 3/4', 'BATHDESC_UPPER LEVEL 1/2 BATH', 'BATHDESC_ROUGH IN', 'BATHDESC_UPPER LEVEL FULL BATH', 'BATHDESC_1/2 MASTER', 'BATHDESC_1/2 BASEMENT', 'BATHDESC_JACK AND JILL', 'BATHDESC_UPPER LEVEL 3/4 BATH', 'ZONING_INDUSTRIAL', 'ZONING_BUSINESS/COMMERCIAL', 'ZONING_OTHER', 'ZONING_RESIDENTIAL-SINGLE', 'ZONING_RESIDENTIAL-MULTI-FAMILY', 'COOLINGDESCRIPTION_WINDOW', 'COOLINGDESCRIPTION_WALL', 'COOLINGDESCRIPTION_DUCTLESS MINI-SPLIT', 'COOLINGDESCRIPTION_NONE', 'COOLINGDESCRIPTION_GEOTHERMAL', 'COOLINGDESCRIPTION_CENTRAL', 'CITY:LELM - LAKE ELMO', 'CITY:MAPW - MAPLEWOOD', 'CITY:OAKD - OAKDALE', 'CITY:STP - SAINT PAUL', 'CITY:WB - WOODBURY', 'LISTTYPE:EXCLUSIVE AGENCY', 'LISTTYPE:EXCLUSIVE RIGHT', 'LISTTYPE:EXCLUSIVE RIGHT WITH EXCLUSIONS', 'LISTTYPE:OTHER', 'LISTTYPE:SERVICE AGREEMENT', 'SCHOOLDISTRICTNUMBER:6 - SOUTH ST. PAUL', 'SCHOOLDISTRICTNUMBER:622 - NORTH ST PAUL-MAPLEWOOD', 'SCHOOLDISTRICTNUMBER:623 - ROSEVILLE', 'SCHOOLDISTRICTNUMBER:624 - WHITE BEAR LAKE', 'SCHOOLDISTRICTNUMBER:625 - ST. PAUL', 'SCHOOLDISTRICTNUMBER:832 - MAHTOMEDI', 'SCHOOLDISTRICTNUMBER:833 - SOUTH WASHINGTON COUNTY', 'SCHOOLDISTRICTNUMBER:834 - STILLWATER', 'POTENTIALSHORTSALE:NO', 'POTENTIALSHORTSALE:NOT DISCLOSED', 'STYLE:(CC) CONVERTED MANSION', 'STYLE:(CC) HIGH RISE (4+ LEVELS)', 'STYLE:(CC) LOW RISE (3- LEVELS)', 'STYLE:(CC) MANOR/VILLAGE', 'STYLE:(CC) TWO UNIT', 'STYLE:(SF) FOUR OR MORE LEVEL SPLIT', 'STYLE:(SF) MODIFIED TWO STORY', 'STYLE:(SF) MORE THAN TWO STORIES', 'STYLE:(SF) ONE 1/2 STORIES', 'STYLE:(SF) ONE STORY', 'STYLE:(SF) OTHER', 'STYLE:(SF) SPLIT ENTRY (BI-LEVEL)', 'STYLE:(SF) THREE LEVEL SPLIT', 'STYLE:(SF) TWO STORIES', 'STYLE:(TH) DETACHED', 'STYLE:(TH) QUAD/4 CORNERS', 'STYLE:(TH) SIDE X SIDE', 'STYLE:(TW) TWIN HOME', 'ASSUMABLEMORTGAGE:INFORMATION COMING', 'ASSUMABLEMORTGAGE:NOT ASSUMABLE', 'ASSUMABLEMORTGAGE:YES W/ QUALIFYING', 'ASSUMABLEMORTGAGE:YES W/NO QUALIFYING', 'ASSESSMENTPENDING:NO', 'ASSESSMENTPENDING:UNKNOWN', 'ASSESSMENTPENDING:YES']


In [116]:
df = dfs

In [117]:
obs_threshold = 30
cols_to_remove = list()
# Inspect first 10 binary columns in list
for col in binary_cols[0:10]:
    # Count the number of 1 values in the binary column
    obs_count = df.agg({col: 'sum'}).collect()[0][0]
    # If less than our observation threshold, remove
    if obs_count < obs_threshold:
        cols_to_remove.append(col)

# Drop columns and print starting and ending dataframe shapes
new_df = df.drop(*cols_to_remove)

print('Rows: ' + str(df.count()) + ' Columns: ' + str(len(df.columns)))
print('Rows: ' + str(new_df.count()) + ' Columns: ' + str(len(new_df.columns)))

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `FENCE_WIRE` cannot be resolved. Did you mean one of the following? [`SchoolDistrictNumber`, `PostalCode`, `NO`, `MLSID`, `StreetNumberNumeric`, `streetaddress`, `STREETNAME`, `StateOrProvince`, `City`, `SALESCLOSEPRICE`, `LISTDATE`, `LISTPRICE`, `LISTTYPE`, `OriginalListPrice`, `PricePerTSFT`, `FOUNDATIONSIZE`, `FENCE`, `MapLetter`, `LotSizeDimensions`, `DAYSONMARKET`, `offmarketdate`, `FIREPLACES`, `RoomArea4`, `roomtype`, `ROOF`, `RoomFloor4`, `PotentialShortSale`, `PoolDescription`, `PDOM`, `GarageDescription`, `SQFTABOVEGROUND`, `TAXES`, `RoomFloor1`, `RoomArea1`, `TAXWITHASSESSMENTS`, `TAXYEAR`, `LIVINGAREA`, `UNITNUMBER`, `YEARBUILT`, `ZONING`, `STYLE`, `ACRES`, `CoolingDescription`, `APPLIANCES`, `BACKONMARKETDATE`, `ROOMFAMILYCHAR`, `RoomArea3`, `EXTERIOR`, `RoomFloor3`, `RoomFloor2`, `RoomArea2`, `DiningRoomDescription`, `BASEMENT`, `BATHSFULL`, `BATHSHALF`, `BATHQUARTER`, `BATHSTH

In [124]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

df = df.withColumnRenamed('City', 'CITY')
categorical_cols = ['CITY', 'LISTTYPE', 'SCHOOLDISTRICTNUMBER', 'POTENTIALSHORTSALE', 'STYLE', 'ASSUMABLEMORTGAGE', 'ASSESSMENTPENDING']
print([col for col in categorical_cols if col not in df.columns])
# Replace missing values
df = df.fillna(-1, subset=['WALKSCORE', 'BIKESCORE'])

# Create list of StringIndexers using list comprehension
indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_IDX").setHandleInvalid("keep")
        for col in categorical_cols
]
# Create pipeline of indexers
indexer_pipeline = Pipeline(stages=indexers)
# Fit and Transform the pipeline to the original data
df_indexed = indexer_pipeline.fit(df).transform(df)

# Clean up redundant columns
df_indexed = df_indexed.drop(*categorical_cols)
# Inspect data transformations
print(df_indexed.dtypes)

['SCHOOLDISTRICTNUMBER', 'POTENTIALSHORTSALE', 'ASSUMABLEMORTGAGE']


Py4JJavaError: An error occurred while calling o1338.fit.
: org.apache.spark.SparkException: Input column SCHOOLDISTRICTNUMBER does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:293)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:290)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115

In [125]:
from pyspark.ml.regression import GBTRegressor

# Train a Gradient Boosted Trees (GBT) model.
gbt = GBTRegressor(
    featuresCol='features',
    labelCol='SALESCLOSEPRICE',
    predictionCol="Prediction_Price",
    seed=42
)

# Train model.
model = gbt.fit(train_df)

IllegalArgumentException: features does not exist. Available: DAYSONMARKET, LISTDATE, OFFMKTDATE

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

# Select columns to compute test error
evaluator = RegressionEvaluator(
	labelCol="SALESCLOSEPRICE",
	predictionCol="Prediction_Price"
)
# Dictionary of model predictions to loop over
models = {
	'Gradient Boosted Trees': gbt_predictions,
	'Random Forest Regression': rfr_predictions
}
for key, preds in models.items():
    # Create evaluation metrics
    rmse = evaluator.evaluate(preds, {evaluator.metricName: "rmse"})
    r2 = evaluator.evaluate(preds, {evaluator.metricName: "r2"})

    # Print Model Metrics
    print(key + ' RMSE: ' + str(rmse))
    print(key + ' R^2: ' + str(r2))

In [ ]:
importances = model.featureImportances.toArray()
# Convert feature importances to a pandas column
fi_df = pd.DataFrame(importances, columns=['importance'])

feature_cols = ['FOUNDATIONSIZE', 'DAYSONMARKET', 'FIREPLACES', 'SQFTABOVEGROUND', 'TAXES', 'TAXWITHASSESSMENTS', 'LIVINGAREA', 'YEARBUILT', 'ACRES', 'BATHSFULL', 'BATHSHALF', 'BATHQUARTER', 'BATHSTHREEQUARTER', 'BATHSTOTAL', 'BEDROOMS', 'SQFTBELOWGROUND', 'ASSOCIATIONFEE', 'ASSESSEDVALUATION', 'LIST_YEAR', 'LIST_MONTH', 'LIST_WEEKOFYEAR', 'LIST_DAYOFMONTH', 'LIST_DAYOFYEAR', 'WALKSCORE', 'BIKESCORE', 'MORTGAGE30US', 'MORTGAGE30US-1WK', 'MORTGAGE30US-2WK', 'MORTGAGE30US-3WK', 'MORTGAGE30US-4WK', 'PERCENT_OLDER_AGE_HOMES', 'PERCENT_SIMILAR_AGE_HOMES', 'PERCENT_NEWER_AGE_HOMES', 'PERCENT_BIGGER_SIZE_HOMES', 'PERCENT_SIMILAR_SIZE_HOMES', 'PERCENT_SMALLER_SIZE_HOMES', 'SQFT_TOTAL', 'BED_TO_BATHS', 'FENCE_OTHER', 'FENCE_WIRE', 'FENCE_PARTIAL', 'FENCE_CHAINLINK', 'FENCE_FULL', 'FENCE_PRIVACY', 'FENCE_NONE', 'FENCE_WOOD', 'ROOF_AGEOVER8YEARS', 'ROOF_ASPHALTSHINGLES', 'ROOF_AGE8YEARSORLESS', 'ROOF_FLAT', 'POOLDESCRIPTION_SHARED', 'POOLDESCRIPTION_BELOWGROUND', 'POOLDESCRIPTION_NONE', 'GARAGEDESCRIPTION_HEATEDGARAGE', 'GARAGEDESCRIPTION_TUCKUNDER', 'GARAGEDESCRIPTION_UNCOVERED/OPEN', 'GARAGEDESCRIPTION_OTHER', 'GARAGEDESCRIPTION_ATTACHEDGARAGE', 'GARAGEDESCRIPTION_DRIVEWAY-ASPHALT', 'GARAGEDESCRIPTION_DETACHEDGARAGE', 'GARAGEDESCRIPTION_NONE', 'GARAGEDESCRIPTION_UNDERGROUNDGARAGE', 'APPLIANCES_COOKTOP', 'APPLIANCES_RANGE', 'EXTERIOR_BRICK/STONE', 'EXTERIOR_CEMENTBOARD', 'EXTERIOR_OTHER', 'EXTERIOR_STUCCO', 'EXTERIOR_METAL', 'EXTERIOR_FIBERBOARD', 'EXTERIOR_SHAKES', 'EXTERIOR_WOOD', 'EXTERIOR_VINYL', 'DININGROOMDESCRIPTION_SEPARATE/FORMALDININGROOM', 'DININGROOMDESCRIPTION_INFORMALDININGROOM', 'DININGROOMDESCRIPTION_BREAKFASTAREA', 'DININGROOMDESCRIPTION_EATINKITCHEN', 'DININGROOMDESCRIPTION_LIVING/DININGROOM', 'DININGROOMDESCRIPTION_KITCHEN/DININGROOM', 'BASEMENT_WALKOUT', 'BASEMENT_CRAWLSPACE', 'BASEMENT_PARTIALFINISHED', 'BASEMENT_PARTIAL', 'BASEMENT_UNFINISHED', 'BASEMENT_FULL', 'BASEMENT_NONE', 'BASEMENT_SLAB', 'BATHDESC_MAINFLOORFULLBATH', 'BATHDESC_UPPERLEVELFULLBATH', 'BATHDESC_MAINFLOOR1/2BATH', 'BATHDESC_MAINFLOOR3/4BATH', 'ZONING_RESIDENTIAL-SINGLE', 'ZONING_RESIDENTIAL-MULTI-FAMILY', 'COOLINGDESCRIPTION_WALL', 'COOLINGDESCRIPTION_DUCTLESSMINI-SPLIT', 'COOLINGDESCRIPTION_WINDOW', 'COOLINGDESCRIPTION_CENTRAL', 'COOLINGDESCRIPTION_NONE', 'CITY_LELM-LAKEELMO', 'CITY_MAPW-MAPLEWOOD', 'CITY_OAKD-OAKDALE', 'CITY_STP-SAINTPAUL', 'CITY_WB-WOODBURY', 'LISTTYPE_EXCLUSIVERIGHT', 'LISTTYPE_SERVICEAGREEMENT', 'SCHOOLDISTRICTNUMBER_622-NORTHSTPAUL-MAPLEWOOD', 'SCHOOLDISTRICTNUMBER_623-ROSEVILLE', 'SCHOOLDISTRICTNUMBER_625-STPAUL', 'SCHOOLDISTRICTNUMBER_833-SOUTHWASHINGTONCOUNTY', 'SCHOOLDISTRICTNUMBER_834-STILLWATER', 'POTENTIALSHORTSALE_NO', 'STYLE_CCHIGHRISE4+LEVELS', 'STYLE_CCLOWRISE3-LEVELS', 'STYLE_CCMANOR/VILLAGE', 'STYLE_SFFOURORMORELEVELSPLIT', 'STYLE_SFMODIFIEDTWOSTORY', 'STYLE_SFMORETHANTWOSTORIES', 'STYLE_SFONE1/2STORIES', 'STYLE_SFONESTORY', 'STYLE_SFSPLITENTRYBI-LEVEL', 'STYLE_SFTHREELEVELSPLIT', 'STYLE_SFTWOSTORIES', 'STYLE_THDETACHED', 'STYLE_THQUAD/4CORNERS', 'STYLE_THSIDEXSIDE', 'STYLE_TWTWINHOME', 'ASSUMABLEMORTGAGE_NOTASSUMABLE', 'ASSESSMENTPENDING_NO', 'ASSESSMENTPENDING_UNKNOWN', 'ASSESSMENTPENDING_YES']

# Convert list of feature names to pandas column
fi_df['feature'] = pd.Series(feature_cols)

# Sort the data based on feature importance
fi_df.sort_values(by=['importance'], ascending=False, inplace=True)

# Inspect Results
fi_df.head(10)

In [ ]:
from pyspark.ml.regression import RandomForestRegressionModel

# Save model
model.save('rfr_no_listprice')

# Load model
loaded_model = RandomForestRegressionModel.load('rfr_no_listprice')